In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
df = pd.read_csv("/content/drive/MyDrive/active_passive_full_cleaned.tsv", sep="\t", header=None)

df.columns = ["Passive", "Active"]

In [5]:
df.head()

,Passive,Active
0,In cases of dispute the matter shall be resolv...,In cases of dispute the judgement of the twent...
1,A dispute on this point shall be determined in...,The judgement of equals shall determine in the...
2,This matter shall be resolved by the judgement...,The judgement of his equals shall resolve this...
3,"If any Freeman shall die Intestate, his chatte...","If any Freeman shall die Intestate, the hands ..."
4,All evil customs connected with forests and Wa...,Twelve sworn Knights of the same County chosen...


In [6]:
!pip uninstall -y transformers
!pip install transformers
!pip install accelerate -U

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached 

In [7]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-act2pas",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [9]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


In [10]:
from evaluate import load
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9871ff72aba85c532c3e5c1b8a33bf2eb18bc1e4e4d0711ca8249c999e35ef80
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [11]:
metric1 = load("rouge")

In [12]:
metric2 = load("bleu")

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [14]:
df

,Passive,Active
0,In cases of dispute the matter shall be resolv...,In cases of dispute the judgement of the twent...
1,A dispute on this point shall be determined in...,The judgement of equals shall determine in the...
2,This matter shall be resolved by the judgement...,The judgement of his equals shall resolve this...
3,"If any Freeman shall die Intestate, his chatte...","If any Freeman shall die Intestate, the hands ..."
4,All evil customs connected with forests and Wa...,Twelve sworn Knights of the same County chosen...
...,...,...
472630,She was led into the church by an adopted daug...,"An adopted daughter, whom she had rescued from..."
472631,Such a servant of the country should be well p...,The country well should pay such a servant of ...
472632,Harriet was sent to me from Boston by Governor...,"Governor Andrew, of Massachusetts sent to me f..."
472633,"She was employed by General Hunter, and I thin...","General Hunter employed her, and I think by ge..."


In [15]:
df2 = pd.read_csv("/content/drive/MyDrive/active_passive.csv")


In [16]:
df2.head()

,Active,Passive
0,He reads a novel.,A novel is read.
1,He does not cook food.,Food is not cooked by him.
2,Does he purchase books?,Are books being purchased by him?
3,They grow plants.,Plants are grown by them.
4,She teaches me.,I am taught by her.


In [17]:
df1 = pd.concat([df2, df], ignore_index=True)

In [18]:
df1

,Active,Passive
0,He reads a novel.,A novel is read.
1,He does not cook food.,Food is not cooked by him.
2,Does he purchase books?,Are books being purchased by him?
3,They grow plants.,Plants are grown by them.
4,She teaches me.,I am taught by her.
...,...,...
472670,"An adopted daughter, whom she had rescued from...",She was led into the church by an adopted daug...
472671,The country well should pay such a servant of ...,Such a servant of the country should be well p...
472672,"Governor Andrew, of Massachusetts sent to me f...",Harriet was sent to me from Boston by Governor...
472673,"General Hunter employed her, and I think by ge...","She was employed by General Hunter, and I thin..."


In [19]:
df1['Active_word_count'] = df1['Active'].apply(lambda x: len(x.split()))

# Filter out rows where the number of words in "Active" is more than 20
df_filtered = df1[df1['Active_word_count'] <= 15]

# Drop the temporary column 'Active_word_count'
df1 = df_filtered.drop(columns=['Active_word_count'])

In [20]:
df1

,Active,Passive
0,He reads a novel.,A novel is read.
1,He does not cook food.,Food is not cooked by him.
2,Does he purchase books?,Are books being purchased by him?
3,They grow plants.,Plants are grown by them.
4,She teaches me.,I am taught by her.
...,...,...
472667,A roar of satisfaction from the crowd below re...,This was responded to by a roar of satisfactio...
472669,Various names among her Southern friends knew ...,Harriet was known by various names among her S...
472671,The country well should pay such a servant of ...,Such a servant of the country should be well p...
472672,"Governor Andrew, of Massachusetts sent to me f...",Harriet was sent to me from Boston by Governor...


In [21]:
max_input_length = 128
max_target_length = 128

def preprocess_function(row):
    inputs = row["Active"] + "</s>"
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=row["Passive"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
tokenized__data = df1.apply(preprocess_function, axis=1)


In [24]:
tokenized__data

0         [input_ids, attention_mask, labels]
1         [input_ids, attention_mask, labels]
2         [input_ids, attention_mask, labels]
3         [input_ids, attention_mask, labels]
4         [input_ids, attention_mask, labels]
                         ...                 
472667    [input_ids, attention_mask, labels]
472669    [input_ids, attention_mask, labels]
472671    [input_ids, attention_mask, labels]
472672    [input_ids, attention_mask, labels]
472674    [input_ids, attention_mask, labels]
Length: 172372, dtype: object

In [25]:
from transformers import AutoModelForSeq2SeqLM


In [26]:
from sklearn.model_selection import train_test_split

# Step 1: Split the data into train and combined validation/test sets
train_data, val_test_data = train_test_split(tokenized__data, test_size=0.2, random_state=42)

# Step 2: Further split the combined validation/test set into separate validation and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=42)

# Now you have train_data, val_data, and test_data


In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [28]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric1.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [29]:
train_dataset = train_data.to_frame().to_dict(orient='records')
val_dataset = val_data.to_frame().to_dict(orient='records')

In [30]:
test_dataset=test_data.to_frame().to_dict(orient='records')

In [31]:
train_dataset1 = [v for d in train_dataset for k, v in d.items()]
val_dataset1=[v for d in val_dataset for k, v in d.items()]

In [32]:
print(train_dataset1[:10])

[{'input_ids': [466, 689, 13, 5464, 470, 780, 28648, 135, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [328, 130, 470, 780, 28648, 57, 24, 689, 13, 5464, 5, 1]}, {'input_ids': [86, 3, 9, 360, 3978, 3, 9, 3, 1007, 13, 4840, 9700, 179, 5575, 15, 7, 3, 8623, 5708, 7, 15416, 9, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [86, 3, 9, 360, 3978, 5708, 7, 15416, 9, 47, 3, 8623, 57, 3, 9, 3, 1007, 13, 4840, 9700, 179, 5575, 15, 7, 5, 1]}, {'input_ids': [299, 34, 59, 2139, 8, 1137, 13, 7332, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [299, 8, 1137, 13, 7332, 47, 59, 2139, 57, 34, 5, 1]}, {'input_ids': [37, 7387, 1162, 3, 23477, 7, 107, 21979, 7, 3, 29532, 8, 851, 13, 8, 1148, 1726, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [37, 851, 13, 8, 1148, 1726, 47, 3, 29532, 57, 8, 7387, 1162, 3, 23477, 7, 107, 21979, 7, 

In [33]:
from huggingface_hub import notebook_login

notebook_login()

In [34]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset1 ,
    eval_dataset=val_dataset1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
print("hello")

hello


In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.166800,0.120069,96.566000,93.685600,94.946300,94.956700,16.542700
2,0.143400,0.106744,96.680800,94.086200,95.209500,95.223200,16.547400
3,0.129000,0.103593,96.719600,94.174600,95.298600,95.312900,16.546600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=25857, training_loss=0.17705067936540658, metrics={'train_runtime': 3737.0432, 'train_samples_per_second': 110.7, 'train_steps_per_second': 6.919, 'total_flos': 2851291726675968.0, 'train_loss': 0.17705067936540658, 'epoch': 3.0})

In [38]:
trainer.save_model("/content/drive/MyDrive/active")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1712682796.23eb4df8627a.424.0:   0%|          | 0.00/18.4k [00:00<?, ?B/s]

In [39]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.10359329730272293,
 'eval_rouge1': 96.7196,
 'eval_rouge2': 94.1746,
 'eval_rougeL': 95.2986,
 'eval_rougeLsum': 95.3129,
 'eval_gen_len': 16.5466,
 'eval_runtime': 331.7764,
 'eval_samples_per_second': 51.954,
 'eval_steps_per_second': 3.249,
 'epoch': 3.0}

In [54]:
test_dataset1=[v for d in test_dataset for k, v in d.items()]


{'input_ids': [100, 97, 6, 983, 6, 8, 4595, 2448, 8238, 8, 10783, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [100, 97, 6, 983, 6, 8, 10783, 47, 4335, 57, 8, 4595, 2448, 5, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]}

In [61]:
# Filter out padding tokens (-100) from labels
filtered_labels = [label for label in test_dataset1[0]['labels'] if label >= 0]

# Decode the filtered labels
decoded_labels = tokenizer.decode(filtered_labels, skip_special_tokens=True)
print(decoded_labels)


This time, however, the spell was broken by the bear himself.


In [77]:
a=test_dataset1[0]['input_ids']
a

[100, 97, 6, 983, 6, 8, 4595, 2448, 8238, 8, 10783, 5, 1, 1]

In [85]:
a = test_dataset1[0]['input_ids']
input_features = {'input_ids': a}
test_predictions = trainer.predict([input_features])
t=test_predictions.predictions
test_predictions

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


PredictionOutput(predictions=array([[    0,   100,    97,     6,   983,     6,     8, 10783,    47,
         4335,    57,     8,  4595,  2448,     5,     1,     0,     0,
            0,     0]]), label_ids=None, metrics={'test_runtime': 0.1888, 'test_samples_per_second': 5.297, 'test_steps_per_second': 5.297})

In [86]:
tokenizer.decode(t[0], skip_special_tokens=True)

'This time, however, the spell was broken by the bear himself.'

In [64]:
predicted_sentences = []
for prediction in test_predictions.predictions:
    tokens = tokenizer.decode(prediction, skip_special_tokens=True)
    predicted_sentences.append(tokens)



# Print the predicted sentences

for i, sentence in enumerate(predicted_sentences):
    filtered_labels = [label for label in test_dataset1[0]['labels'] if label >= 0]

# Decode the filtered labels
    decoded_labels = tokenizer.decode(filtered_labels, skip_special_tokens=True)

    print(f"Example {i+1} - {decoded_labels} Predicted Sentence: {sentence}")


Example 1 - This time, however, the spell was broken by the bear himself. Predicted Sentence: This time, however, the spell was broken by the bear himself.
Example 2 - This time, however, the spell was broken by the bear himself. Predicted Sentence: At that time, under treaty provisions, the neutrality of Hawaii was guaranteed by the United
Example 3 - This time, however, the spell was broken by the bear himself. Predicted Sentence: It was broken by crevices and strewn with Bowlders,
Example 4 - This time, however, the spell was broken by the bear himself. Predicted Sentence: They are much influenced by rainfall.
Example 5 - This time, however, the spell was broken by the bear himself. Predicted Sentence: She was buried by night at Westminster.
Example 6 - This time, however, the spell was broken by the bear himself. Predicted Sentence: The King was served by the Lords and pensioners who brought up the dishes.
Example 7 - This time, however, the spell was broken by the bear himself. Pr

In [73]:
test_dataset1[0]

{'input_ids': [100, 97, 6, 983, 6, 8, 4595, 2448, 8238, 8, 10783, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [100, 97, 6, 983, 6, 8, 10783, 47, 4335, 57, 8, 4595, 2448, 5, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]}

In [97]:
input_text = "I am doing my work "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(input_ids)

# Convert input_ids tensor to a dictionary
input_features = {"input_ids": input_ids[0]}

# Generate predictions
output = trainer.predict([input_features])
output
tokenizer.decode(output.predictions[0], skip_special_tokens=True)
# input_features

tensor([[ 27, 183, 692,  82, 161,   1]])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'My work is done by me.'

In [36]:
print("hello")

hello


In [98]:
from transformers import AutoModelForSeq2SeqLM
output_dir="/content/drive/MyDrive/active"
# Load the saved model
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir)

In [99]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Tokenize input text
input_text = "I am doing my work"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate predictions
output = loaded_model.generate(input_ids)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[  0, 499, 161,  19, 612,  57, 140,   5,   1]])

In [100]:
b=output.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)

'My work is done by me.'

In [101]:
trainer.push_to_hub("Pushparaj2811/active")

events.out.tfevents.1712687120.23eb4df8627a.424.1:   0%|          | 0.00/623 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Pushparaj2811/t5-small-act2pas/commit/38ae9fac06de4a526d51d237082c2dce954f6bb6', commit_message='Pushparaj2811/active', commit_description='', oid='38ae9fac06de4a526d51d237082c2dce954f6bb6', pr_url=None, pr_revision=None, pr_num=None)

In [112]:
from transformers import AutoModelForSeq2SeqLM

# Load the model from the model hub
model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj2811/t5-small-act2pas")

# Now you can use the loaded model for inference or fine-tuning
input_text = "I am doing my work"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate predictions
outpu = loaded_model.generate(input_ids)
outpu

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[  0, 499, 161,  19, 612,  57, 140,   5,   1]])

In [113]:
b=outpu.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)

'My work is done by me.'

In [108]:
m=AutoModelForSeq2SeqLM.from_pretrained("Pushparaj2811/t5-small-act2pas")

In [111]:
df2.shape

(40, 2)

In [115]:
for i in range((df2.shape)[0]):
  input_ids = tokenizer(df2['Active'][i], return_tensors="pt").input_ids

# Generate predictions
  outpu = loaded_model.generate(input_ids)
  b=outpu.numpy()
  print(df2['Active'][i],df2['Passive'][i],tokenizer.decode(b[0], skip_special_tokens=True))




He reads a novel. A novel is read. A novel is read by him.
He does not cook food. Food is not cooked by him. Food is not cooked by him.
Does he purchase books? Are books being purchased by him? Does he purchase books?
They grow plants. Plants are grown by them. Plants are grown by them.
She teaches me. I am taught by her. I am taught by her.
Esha is singing a song. A song is being sung by Esha. A song is being sung by Esha.
Kritika is not chopping vegetables. Vegetables are not being chopped by Kritika. Vegetables are not chopping by Kritika.
Is Ritika buying a table? Is a table being bought by Ritika? Is Ritika buying a table?
They are serving poor people. Poor people are being served by them. Poor people are being served by them.
She is disturbing Dinesh. Dinesh is being disturbed by her. Dinesh is being disturbed by her.
Nitesh has challenged her. She has been challenged by Nitesh. She has been challenged by Nitesh.
Radhika has not written an article. An article has not been written

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[  0, 499, 161,  19, 612,  57, 140,   5,   1]])